# Spotify Databases Demo

### Imports

In [22]:
import data_cleaning
import tabular_query
import graph_query

import time
import redis
import warnings
import psycopg2
from psycopg2 import extras
from neo4j import GraphDatabase
from cassandra.cluster import Cluster
warnings.filterwarnings("ignore")

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Cleaning

In [24]:
artist_df, track_df= data_cleaning.clean_data_tabular()
artist_nodes, track_nodes, collaborations = data_cleaning.clean_data_graph(track_df)
for df in [artist_df, track_df, artist_nodes, track_nodes, collaborations]:
    display(df.head(1))

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,demo
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.445,...,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,3,False


,spotify_id,name,followers,popularity,genres,chart_hits
1,4lDiJcOJ2GLCK6p9q5BgfK,Kontra K,1999676.0,72,"['christlicher rap', 'german hip hop']","['at (44)', 'de (111)', 'lu (22)', 'ch (31)', ..."


,id,name,popularity,duration_ms,artists,id_artists,danceability,energy,key,loudness,liveness,tempo
150,3eMrYc092k7SIJfWJ7oasR,Weather Bird,38,161933,"['Louis Armstrong','Earl Hines']","19eLuQmk9aCobbVDHc6eek,2mY5u4CceAPrpBnse1WpFr",0.831,0.262,8,-12.386,0.204,104.606


,id_0,id_1
0,76M2Ekj8bG8W7X2nbx2CpF,7sfl4Xt5KmfyDs2T3SVSMK


### PostgreSQL with Redis

Redis setup

In [4]:
# Redis
r = redis.Redis(host='localhost', port=6379, db=0,
                decode_responses=True)
r.hmset('fact_memroy', {0: 1, 1: 1, 'max': 1})
r.flushall()

True

PostgreSQL setup

In [5]:
# PostgreSQL
conn = psycopg2.connect(dbname='my_database', user='postgres',
                        password='mysecretpassword', host='localhost',
                        port='5433')
cur = conn.cursor()

In [6]:
# Drop and create table artist_df 
drop_table = """
DROP TABLE IF EXISTS artist_df
"""
cur.execute(drop_table)

create_table = """
    CREATE TABLE artist_df (
        id TEXT,
        followers INT,
        genres TEXT,
        name TEXT,
        popularity INT,
        PRIMARY KEY (id)
    );
"""
cur.execute(create_table)

In [7]:
artist_data = artist_df.values.tolist()
insertion = """
    INSERT INTO artist_df 
    (id, followers, genres, name, popularity)
    VALUES %s
"""
extras.execute_values(cur, insertion, artist_data)
conn.commit()

##### Example 1: 
In this example, I retrieve information about the Canadian pop singer Justin Bieber. Initially, there is no cache present in Redis, so the first retrieval of this query takes longer compared to the second retrieval. The benefit of Redis for this function is to reduce the time spent by users on repetitive queries.

In [8]:
start = time.time()
result = tabular_query.search_artist(r, cur, name = 'Justin Bieber', projection = ['id', 'name', 'followers'])
end = time.time()

before_redis = end - start
print(f"PostgreSQL not using Redis as cache, time: {end - start} s.")
print(result)
print()

start = time.time()
result = tabular_query.search_artist(r, cur, name = 'Justin Bieber', projection = ['id', 'name', 'followers'])
end = time.time()
after_redis = end - start
print(f"PostgreSQL using Redis as cache, time: {end - start} s.")
print(result)

print('----------------------------------------------------------------------------------------')
print(f'Speed Up :{before_redis/after_redis}')

Query: 
        SELECT
        id , name , followers
        FROM artist_df
        WHERE name = 'Justin Bieber'
        
    
PostgreSQL not using Redis as cache, time: 0.08336782455444336 s.
[('1uNFoZAHBGtllmzznpCI3s', 'Justin Bieber', 44606973)]

Query: 
        SELECT
        id , name , followers
        FROM artist_df
        WHERE name = 'Justin Bieber'
        
    
PostgreSQL using Redis as cache, time: 0.0009121894836425781 s.
[('1uNFoZAHBGtllmzznpCI3s', 'Justin Bieber', 44606973)]
----------------------------------------------------------------------------------------
Speed Up :91.39309984317825


##### Example 2: 
In this example, our function attempts to retrieve information about the renowned Indian singer Udit Narayan. However, this time we do not provide any information regarding the projection step. In other words, we only provide information about the selection step, specifically the 'name' column. When we do not provide information about the projection step, the function returns all columns.

In [9]:
start = time.time()
result = tabular_query.search_artist(r, cur, name = 'Udit Narayan')
end = time.time()

before_redis = end - start
print(f"PostgreSQL not using Redis as cache, time: {end - start} s.")
print(result)
print()

start = time.time()
result = tabular_query.search_artist(r, cur, name = 'Udit Narayan')
end = time.time()
after_redis = end - start
print(f"PostgreSQL using Redis as cache, time: {end - start} s.")
print(result)

print('----------------------------------------------------------------------------------------')
print(f'Speed Up :{before_redis/after_redis}')

Query: 
        SELECT
        id , followers , genres , name , popularity
        FROM artist_df
        WHERE name = 'Udit Narayan'
        
    
PostgreSQL not using Redis as cache, time: 0.06472110748291016 s.
[('70B80Lwx2sxti0M1Ng9e8K', 3409497, "['chutney','classic bollywood','desi pop','filmi','sufi']", 'Udit Narayan', 72)]

Query: 
        SELECT
        id , followers , genres , name , popularity
        FROM artist_df
        WHERE name = 'Udit Narayan'
        
    
PostgreSQL using Redis as cache, time: 0.0013632774353027344 s.
[('70B80Lwx2sxti0M1Ng9e8K', 3409497, "['chutney','classic bollywood','desi pop','filmi','sufi']", 'Udit Narayan', 72)]
----------------------------------------------------------------------------------------
Speed Up :47.474641483036024


### Cassandra with Redis

Cassandra setup

In [10]:
# Cassandra
cluster = Cluster(['127.0.0.1'])
cassandra_session = cluster.connect()

cassandra_session.execute("""
    create keyspace if not exists cassandra_redis
    with replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")
cassandra_session.set_keyspace('cassandra_redis')

#### Function ①: popularity_by_month(year, month)
In this function, users can input the year and month to obtain the  number of tracks of various popularity levels for that specific month.

Insert data

In [11]:
cassandra_session.execute('DROP TABLE IF EXISTS month_popularity;')

creation = \
    """
    CREATE TABLE IF NOT EXISTS month_popularity (
        year INT,
        month INT,
        popularity INT,
        id TEXT,
        PRIMARY KEY ((year, month), popularity, id)
    )
    WITH CLUSTERING ORDER BY (popularity ASC, id ASC);
"""
cassandra_session.execute(creation)
tabular_query.insertion_popularity_by_month(cassandra_session, track_df)

##### Example: 
In this example, our function creates a CQL query based on the user's inputs to retrieve data from Cassandra. We want to examine the distribution of popularity count in January 2015. In the first query, the query cache in Redis does not have the result for our query, so the efficiency is lower. However, in the second query, Redis already has the result cached, so the efficiency is higher.

In [21]:
r.flushall()

start = time.time()
result = tabular_query.popularity_by_month(r, cassandra_session, 2015, 1)
end = time.time()

before_redis = end - start
print(f"Cassandra not using Redis as cache, time: {end - start} s.")

start = time.time()
result = tabular_query.popularity_by_month(r, cassandra_session, 2015, 1)
end = time.time()
after_redis = end - start
print(f"Cassandra using Redis as cache, time: {end - start} s.")

print('----------------------------------------------------------------------------------------')
print(f'Speed Up :{before_redis/after_redis}', '\n')

for line in result[:10]:
    print(line)

Cassandra not using Redis as cache, time: 0.2698540687561035 s.
Cassandra using Redis as cache, time: 0.0019979476928710938 s.
----------------------------------------------------------------------------------------
Speed Up :135.0656324582339 

Row(year=2015, month=1, popularity=0, count=88)
Row(year=2015, month=1, popularity=1, count=28)
Row(year=2015, month=1, popularity=2, count=13)
Row(year=2015, month=1, popularity=3, count=12)
Row(year=2015, month=1, popularity=4, count=10)
Row(year=2015, month=1, popularity=5, count=26)
Row(year=2015, month=1, popularity=6, count=8)
Row(year=2015, month=1, popularity=7, count=9)
Row(year=2015, month=1, popularity=8, count=7)
Row(year=2015, month=1, popularity=9, count=2)


#### Function ②: track_by_music_attributes(key, mode, time_signature, limit)
In this function, users can utilize the track's modality, estimated time signature, and key signature to extract the names of all appropriate songs and their corresponding IDs.

Insert data

In [13]:
cassandra_session.execute('DROP TABLE IF EXISTS music_attributes;')

creation = \
    """
    CREATE TABLE IF NOT EXISTS music_attributes (
        key INT,
        mode INT,
        time_signature INT,
        id TEXT,
        name TEXT,
        PRIMARY KEY ((key, mode, time_signature), id)
    )
    WITH CLUSTERING ORDER BY (id ASC);
"""
cassandra_session.execute(creation)
tabular_query.insertion_music_attributes(cassandra_session, track_df)

##### Example: 
In this example, we are looking for music tracks that are in the key of C# or Db, have a minor mode, and an estimated time signature of three beats per measure. We will limit the returned results to 10. It can be observed that when we use Redis as a cache, its performance significantly outperforms using only Cassandra, despite the fact that Cassandra itself is already highly performant.

In [14]:

r.flushall()

start = time.time()
result = tabular_query.track_by_music_attributes(r, cassandra_session, 4, 0, 3, 10)
end = time.time()

before_redis = end - start
print(f"Cassandra not using Redis as cache, time: {end - start} s.")

start = time.time()
result = tabular_query.track_by_music_attributes(r, cassandra_session, 4, 0, 3, 10)
end = time.time()
after_redis = end - start
print(f"Cassandra using Redis as cache, time: {end - start} s.")

print('----------------------------------------------------------------------------------------')
print(f'Speed Up :{before_redis/after_redis}', '\n')

for line in result:
    print(line)

Cassandra not using Redis as cache, time: 0.013391971588134766 s.
Cassandra using Redis as cache, time: 0.0007891654968261719 s.
----------------------------------------------------------------------------------------
Speed Up :16.96978851963746 

Row(id='000KblXP5csWFFFsD6smOy', name='Podrás')
Row(id='00sVcA5dG61xeJPZjscYtc', name='With Every Heartbeat')
Row(id='00zsFwSnoYLpzx3a1xEPW9', name='Murtunut elämä')
Row(id='01XW7KrY2WDZ5VHCribChr', name='Symphony No. 4 in E Minor,Op. 98: IV. Allegro energico e passionata')
Row(id='01dS4ZvJUbDwwe0YuP52Tr', name='İncelikler')
Row(id='01tirLnUWIOOGh5FwPf1jU', name='Страна чудес')
Row(id='01yTSXSghvy98C7I6kvEPJ', name='Vete de Mí Cuervo Negro')
Row(id='028Eip1qNg2pjGrh7eP736', name='If I Had a Ribbon Bow')
Row(id='02BFGMgS4ymR5RuZGNMKK1', name='Taksim Meydanı')
Row(id='02CtvvKZuAJs9WilNTztM0', name='Pumuckl soll Ordnung lernen - Teil 03')


### Neo4j Query

Neo4j setup

In [15]:
# Setting up neo4j connection
url = "bolt://localhost:7666"
username = "neo4j"
password = "password"
def create_neo4j_session(url, username, password):
    try:
        driver = GraphDatabase.driver(url, auth=(username, password))
        session = driver.session()
        return session
    except Exception as e:
        print(f"Failed to create Neo4j session: {e}")
        return None
    
def delete_all():
    session = create_neo4j_session(url, username, password)
    query = "MATCH (n) DETACH DELETE n"
    result = session.run(query)
    session.close()

delete_all()
neo4j_session = create_neo4j_session(url, username, password)

Insert data

In [16]:
# create Artist nodes
create_nodes_query = '''LOAD CSV WITH HEADERS FROM 'file:///cleaned/artist_nodes.csv' AS row
CALL {
    WITH row
    CREATE (:Artist {
        artist_id: row.spotify_id, 
        name: row.name, 
        followers: row.followers, 
        popularity: row.popularity,
        genres: row.genres,
        community_id: null
        })
} IN TRANSACTIONS'''
neo4j_session.run(create_nodes_query)

# create COLLABORATE edges
create_edges_query = '''LOAD CSV WITH HEADERS FROM "file:///cleaned/collaborations.csv" AS row
CALL {
  WITH row
  MATCH (a: Artist {artist_id: row.id_0})
  MATCH (b: Artist {artist_id: row.id_1})
  CREATE (a)-[:COLLABORATE]->(b)
  CREATE (b)-[:COLLABORATE]->(a)
} IN TRANSACTIONS'''

neo4j_session.run(create_edges_query)

# create Track nodes and COMPOSED_BY edges
create_tracks_query = '''LOAD CSV WITH HEADERS FROM 'file:///cleaned/track_nodes.csv' AS row
CALL {
    WITH row
    MATCH (a:Artist)
    WHERE a.artist_id in SPLIT(row.id_artists, ',')
    CREATE (b:Track {track_id: row.id, 
        track_name: row.name, 
        track_popularity: row.popularity, 
        duration_ms: row.duration_ms,
        danceability: toFloat(row.danceability),
        energy: toFloat(row.energy),
        key: toFloat(row.key),
        loudness: toFloat(row.loudness),
        liveness: toFloat(row.liveness),
        tempo: toFloat(row.tempo)
        })-[:COMPOSED_BY]->(a)        
} IN TRANSACTIONS'''
neo4j_session.run(create_tracks_query)

##### Example

In [17]:
# Initializing the communities using Louvain algorithm
graph_query.louvain_cluster(neo4j_session)

In [18]:
# Get one track id from the cassandra track_by_music_attributes query
result = tabular_query.track_by_music_attributes(r, cassandra_session, key=1, mode=0, time_signature=4)
input_track = graph_query.parse_track_id(result[0])
input_track

Failed to write data to connection IPv4Address(('localhost', 7666)) (ResolvedIPv6Address(('::1', 7666, 0, 0)))


'001f6XLtM53gwKSauiUcKI'

In [19]:
# Find similar tracks with the above query result
graph_query.recommend_track(r, neo4j_session, input_track, limit=10)

["{'track_id': '001f6XLtM53gwKSauiUcKI', 'track_name': 'Tigresa - Remixed Original Album', 'pearsonSimilarity': 1.0}",
 "{'track_id': '3L7XJdSpz64JXGx2HSeDdN', 'track_name': 'Pipoca Moderna - Remixed Original Album', 'pearsonSimilarity': 0.9999953990736841}",
 "{'track_id': '1Hn0iEB1FPfJzJPK49qVtH', 'track_name': 'Mora Na Filosofia', 'pearsonSimilarity': 0.9999774442630288}",
 "{'track_id': '26OTzXhJuLqUNzPo0nJ6ug', 'track_name': 'Desde Que O Samba É Samba', 'pearsonSimilarity': 0.9999759164963357}",
 "{'track_id': '3U1kYZqlXH3ZwD7T3IGhT5', 'track_name': 'Escapulário - Remixed Original Abum', 'pearsonSimilarity': 0.9999756197035858}",
 "{'track_id': '3AelUK9uB5eGtcYmqgX7ps', 'track_name': 'Cajuína', 'pearsonSimilarity': 0.999969716392537}",
 "{'track_id': '2VBN5kSs1fyiE5fnOYeKsG', 'track_name': 'Sampa', 'pearsonSimilarity': 0.9999664273227357}",
 "{'track_id': '4LKORkGLmXfwTtxAvMIc2x', 'track_name': 'Sampa', 'pearsonSimilarity': 0.9999655118354478}",
 "{'track_id': '7sjeeE3NeBizcps3Il8

In [20]:
# r.flushall()
neo4j_session.close()